In [2]:
import geopandas as gpd
from sqlalchemy import create_engine
import pyodbc
import fiona

def convert_fgdb_to_your_sqlserver(fgdb_path, layer_name):
    """
    Convert FGDB to your specific SQL Server instance
    """
    # Your SQL Server connection
    connection_string = f"mssql+pyodbc://@100.103.17.32,1433/SpatialTest?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"

    try:
        # Read from FGDB
        print(f"Reading {layer_name} from {fgdb_path}...")
        gdf = gpd.read_file(fgdb_path, layer=layer_name)

        # Create engine
        engine = create_engine(connection_string)

        # Convert geometry to WKT for SQL Server
        gdf['Shape'] = gdf['geometry'].apply(lambda geom: geom.wkt if geom else None)

        # Drop the original geometry column and rename
        gdf = gdf.drop('geometry', axis=1)

        # Write to SQL Server
        gdf.to_sql(
            name=layer_name,
            con=engine,
            if_exists='replace',
            index=False,
            dtype={'Shape': 'GEOMETRY'}
        )

        # Create spatial index with appropriate bounding box
        with engine.connect() as conn:
            conn.execute(f"""
                CREATE SPATIAL INDEX SIDX_{layer_name}_Shape
                ON {layer_name}(Shape)
                USING GEOMETRY_GRID
                WITH (BOUNDING_BOX = (-71.1, 42.9, -66.9, 47.5))
            """)

        print(f"✓ Successfully converted {layer_name} to SpatialTest database")
        return True

    except Exception as e:
        print(f"✗ Error converting {layer_name}: {e}")
        return False

def list_and_convert_fgdb(fgdb_path):
    """
    List all feature classes and convert selected ones
    """
    try:
        # List all layers in FGDB
        layers = fiona.listlayers(fgdb_path)
        print(f"Found {len(layers)} layers in FGDB:")
        for i, layer in enumerate(layers, 1):
            print(f"  {i}. {layer}")

        # Convert each layer
        for layer in layers:
            convert_fgdb_to_your_sqlserver(fgdb_path, layer)

    except Exception as e:
        print(f"Error processing FGDB: {e}")

# Usage examples:
if __name__ == "__main__":
    # Test with sample FGDB
    fgdb_path = "/Users/brendanhall/GitHub/General_Code/ATFS/FGDB to SQL Server Conversion/esri_ref_data.gdb/esri_ref_data.gdb"

    # List and convert all layers
    list_and_convert_fgdb(fgdb_path)

    # Or convert specific layer
    # convert_fgdb_to_your_sqlserver(fgdb_path, "YourSpecificLayer")

Found 4 layers in FGDB:
  1. states
  2. fedlandp
  3. park_dtl
  4. dtl_st
Reading states from /Users/brendanhall/GitHub/General_Code/ATFS/FGDB to SQL Server Conversion/esri_ref_data.gdb/esri_ref_data.gdb...


/opt/miniconda3/envs/General_Code/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: dtl_cnty layer has a /Users/brendanhall/GitHub/General_Code/ATFS/FGDB to SQL Server Conversion/esri_ref_data.gdb/esri_ref_data.gdb/a0000000a.gdbtable.cdf file whose format is unhandled
  return ogr_read(


✗ Error converting states: (pyodbc.Error) ('HY000', "[HY000] [Microsoft][ODBC Driver 17 for SQL Server]SSPI Provider: Server (MSSQLSvc/macpro-win.tail07861.ts.net\x01:1433@TAIL07861.TS.NET\x01) unknown while looking up 'MSSQLSvc/macpro-win.tail07861.ts.net\x01:1433@TAIL07861.TS.NET\x01' (cached result, timeout in 1200 sec) (851968) (SQLDriverConnect)")
(Background on this error at: https://sqlalche.me/e/20/dbapi)
Reading fedlandp from /Users/brendanhall/GitHub/General_Code/ATFS/FGDB to SQL Server Conversion/esri_ref_data.gdb/esri_ref_data.gdb...


/opt/miniconda3/envs/General_Code/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: dtl_cnty layer has a /Users/brendanhall/GitHub/General_Code/ATFS/FGDB to SQL Server Conversion/esri_ref_data.gdb/esri_ref_data.gdb/a0000000a.gdbtable.cdf file whose format is unhandled
  return ogr_read(
/opt/miniconda3/envs/General_Code/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


✗ Error converting fedlandp: (pyodbc.Error) ('HY000', "[HY000] [Microsoft][ODBC Driver 17 for SQL Server]SSPI Provider: Server (MSSQLSvc/macpro-win.tail07861.ts.net\x01:1433@TAIL07861.TS.NET\x01) unknown while looking up 'MSSQLSvc/macpro-win.tail07861.ts.net\x01:1433@TAIL07861.TS.NET\x01' (cached result, timeout in 1199 sec) (851968) (SQLDriverConnect)")
(Background on this error at: https://sqlalche.me/e/20/dbapi)
Reading park_dtl from /Users/brendanhall/GitHub/General_Code/ATFS/FGDB to SQL Server Conversion/esri_ref_data.gdb/esri_ref_data.gdb...


/opt/miniconda3/envs/General_Code/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: dtl_cnty layer has a /Users/brendanhall/GitHub/General_Code/ATFS/FGDB to SQL Server Conversion/esri_ref_data.gdb/esri_ref_data.gdb/a0000000a.gdbtable.cdf file whose format is unhandled
  return ogr_read(


✗ Error converting park_dtl: (pyodbc.Error) ('HY000', "[HY000] [Microsoft][ODBC Driver 17 for SQL Server]SSPI Provider: Server (MSSQLSvc/macpro-win.tail07861.ts.net\x01:1433@TAIL07861.TS.NET\x01) unknown while looking up 'MSSQLSvc/macpro-win.tail07861.ts.net\x01:1433@TAIL07861.TS.NET\x01' (cached result, timeout in 1198 sec) (851968) (SQLDriverConnect)")
(Background on this error at: https://sqlalche.me/e/20/dbapi)
Reading dtl_st from /Users/brendanhall/GitHub/General_Code/ATFS/FGDB to SQL Server Conversion/esri_ref_data.gdb/esri_ref_data.gdb...


/opt/miniconda3/envs/General_Code/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: dtl_cnty layer has a /Users/brendanhall/GitHub/General_Code/ATFS/FGDB to SQL Server Conversion/esri_ref_data.gdb/esri_ref_data.gdb/a0000000a.gdbtable.cdf file whose format is unhandled
  return ogr_read(


✗ Error converting dtl_st: (pyodbc.Error) ('HY000', "[HY000] [Microsoft][ODBC Driver 17 for SQL Server]SSPI Provider: Server (MSSQLSvc/macpro-win.tail07861.ts.net\x01:1433@TAIL07861.TS.NET\x01) unknown while looking up 'MSSQLSvc/macpro-win.tail07861.ts.net\x01:1433@TAIL07861.TS.NET\x01' (cached result, timeout in 1198 sec) (negative cache) (851968) (SQLDriverConnect)")
(Background on this error at: https://sqlalche.me/e/20/dbapi)
